## Instancias y extensiones

Este capítulo cubrirá varios algoritmos variacionales cuánticos, que incluyen:

- [Solucionador propio cuántico variacional (VQE)](https://arxiv.org/abs/1304.3061)
- [Búsqueda subespacial VQE (SSVQE)](https://arxiv.org/abs/1810.09434)
- [Deflación cuántica variacional (VQD)](https://arxiv.org/abs/1805.08138)
- [Regresión de muestreo cuántico (QSR)](https://arxiv.org/pdf/2012.02338)

Mediante el uso de estos algoritmos, aprenderemos sobre varias ideas de diseño que se pueden incorporar en algoritmos variacionales personalizados, como pesos, penalizaciones, sobremuestreo y submuestreo. Lo alentamos a que experimente con estos conceptos y comparta sus hallazgos con la comunidad.

## Solucionador propio cuántico variacional (VQE)

[VQE](https://arxiv.org/abs/1304.3061) es uno de los algoritmos cuánticos variacionales más utilizados, configurando una plantilla para que se desarrollen otros algoritmos.

![VQE](images/instances_VQE.png)

El diseño de VQE es simple:

- Preparar operadores de referencia $U_R$
    - Partimos del estado $|0\rangle$ y vamos al estado de referencia $|\rho\rangle$
- Aplique la forma variacional $U_V(\vec\theta_{i,j})$ para crear un ansatz $U_A(\vec\theta_{i,j})$
    - Pasamos del estado $|\rho\rangle$ a $U_V(\vec\theta_{i,j})|\rho\rangle = |\psi(\vec\theta_{i,j})\rangle$
- Bootstrap en $i=0$ si tenemos un problema similar (generalmente encontrado a través de simulación clásica o muestreo)
    - Cada optimizador se arrancará de forma diferente, lo que dará como resultado un conjunto inicial de vectores de parámetros $\Theta_0 := \{ {\vec\theta_{0,j} | j \in \mathcal{J}_\text{opt}^0} \}$ (por ejemplo, desde un punto inicial $\vec\theta_0$).
- Evalúa la función de costo $C(\vec\theta_{i,j}) := \langle \psi(\vec{\theta}) | \sombrero{H} | \psi(\vec{\theta})\rangle$ para todos los estados preparados en una computadora cuántica.
- Utilice un optimizador clásico para seleccionar el siguiente conjunto de parámetros $\Theta_{i+1}$.
- Repita el proceso hasta que se alcance la convergencia.

Este es un ciclo de optimización clásico simple donde evaluamos la función de costo. Algunos optimizadores pueden requerir múltiples evaluaciones para calcular un gradiente, determinar la siguiente iteración o evaluar la convergencia.

Aquí está el ejemplo para el siguiente observable:

$$
\hat{O}_1 = 2 II - 2 XX + 3 YY - 3 ZZ,
$$

In [ ]:
from qiskit.circuit.library import TwoLocal
from qiskit.quantum_info import SparsePauliOp
from qiskit import QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator
import numpy as np

# Add your token below
service = QiskitRuntimeService(
    channel="ibm_quantum",
)

def cost_function_vqe(theta):
    observable = SparsePauliOp.from_list([("II", 2), ("XX", -2), ("YY", 3), ("ZZ", -3)])
    reference_circuit = QuantumCircuit(2)
    reference_circuit.x(0)

    variational_form = TwoLocal(
        2,
        rotation_blocks=["rz", "ry"],
        entanglement_blocks="cx",
        entanglement="linear",
        reps=1,
    )
    ansatz = reference_circuit.compose(variational_form)

    backend = service.backend("ibmq_qasm_simulator")
    
    # Use estimator to get the expected values corresponding to each ansatz
    estimator = Estimator(session=backend)
    job = estimator.run(ansatz, observable, theta)
    values = job.result().values

    return values

Podemos usar esta función de costo para calcular parámetros óptimos

In [ ]:
from qiskit.algorithms.optimizers import COBYLA

initial_theta = np.ones(8)
optimizer = COBYLA()

optimizer_result = optimizer.minimize(fun=cost_function_vqe, x0=initial_theta)

optimal_parameters = optimizer_result.x
print(optimal_parameters)

Finalmente, podemos usar nuestros parámetros óptimos para calcular nuestros valores propios mínimos:

In [ ]:
observable = SparsePauliOp.from_list([("II", 2), ("XX", -2), ("YY", 3), ("ZZ", -3)])
reference_circuit = QuantumCircuit(2)
reference_circuit.x(0)

variational_form = TwoLocal(
    2,
    rotation_blocks=["rz", "ry"],
    entanglement_blocks="cx",
    entanglement="linear",
    reps=1,
)
ansatz = reference_circuit.compose(variational_form)
solution = ansatz.bind_parameters(optimal_parameters)

backend = service.backend("ibmq_qasm_simulator")
estimator = Estimator(session=backend)
job = estimator.run(solution, observable)
values = job.result().values

experimental_min_eigenvalue = values[0]
print(experimental_min_eigenvalue)

## Búsqueda subespacial VQE (SSVQE)

[SSVQE](https://arxiv.org/abs/1810.09434) es una variante de VQE que permite obtener los primeros $k$ valores propios de un $\hat{H}$ observable con valores propios {$\lambda_0, \lambda_1,...,\lambda_{N-1}$}, donde $N\geq k$. Sin pérdida de generalidad, asumimos que $\lambda_0&lt;\lambda_1&lt;...&lt;\lambda_{N-1}$. SSQVE presenta una nueva idea al agregar pesos para ayudar a priorizar la optimización del término con el mayor peso.

![SSVQE](images/instances_SSVQE.png)

Para implementar este algoritmo, necesitamos $k$ estados de referencia mutuamente ortogonales `{latex} \{ |\rho_j\rangle \}_{j=0}^{k-1}` , lo que significa $\langle \rho_j | \rho_l \rangle = \delta_{jl}$ para $j,l&lt;k$. Estos estados se pueden construir utilizando operadores de Pauli. La función de costo de este algoritmo es entonces:

$$
\begin{aligned}
C(\vec{\theta})

&amp; := \sum_{j=0}^{k-1} w_j \langle \rho_j | U_{V}^{\dagger}(\vec{\theta})\hat{H} U_{V}(\vec{\theta})|\rho_j \rangle \\[1mm]

&amp; := \sum_{j=0}^{k-1} w_j \langle \psi_{j}(\vec{\theta}) | \hat{H} | \psi_{j}(\vec{\theta}) \rangle \\[1mm]

\end{aligned}
$$

donde $w_j$ es un número positivo arbitrario tal que si $j&lt;l&lt;k$ entonces $w_j&gt;w_l$, y $U_V(\vec{\theta})$ es la forma variacional definida por el usuario.

El algoritmo SSVQE se basa en el hecho de que los estados propios correspondientes a diferentes valores propios son mutuamente ortogonales. Específicamente, el producto interno de $U_V(\vec{\theta})|\rho_j\rangle$ y $U_V(\vec{\theta})|\rho_l\rangle$ se puede expresar como:

$$
\begin{aligned}
\langle \rho_j | U_{V}^{\dagger}(\vec{\theta})U_{V}(\vec{\theta})|\rho_l \rangle

&amp; = \langle \rho_j | I |\rho_l \rangle \\[1mm]

&amp; = \langle \rho_j | \rho_l \rangle \\[1mm]

&amp; = \delta_{jl}

\end{aligned}
$$

La primera igualdad se cumple porque $U_{V}(\vec{\theta})$ es un operador cuántico y por lo tanto es unitario. La última igualdad se cumple debido a la ortogonalidad de los estados de referencia $|\rho_j\rangle$. El hecho de que la ortogonalidad se mantenga a través de transformaciones unitarias está profundamente relacionado con el principio de conservación de la información, tal como se expresa en la ciencia de la información cuántica. Bajo este punto de vista, las transformaciones no unitarias representan procesos donde la información se pierde o se inyecta.

Los pesos $w_j$ ayudan a garantizar que todos los estados sean estados propios. Si los pesos son lo suficientemente diferentes, el término con el mayor peso (es decir, $w_0$) tendrá prioridad durante la optimización sobre los demás. Como resultado, el estado resultante $U_{V}(\vec{\theta})|\rho_0 \rangle$ se convertirá en el estado propio correspondiente a $\lambda_0$. `{latex} \{ U_{V}(\vec{\theta})|\rho_j\rangle \}_{j=0}^{k-1}` , por tanto, contenida en el subespacio correspondiente a los valores propios {$\lambda_1,...,\lambda_{N-1}$}.

Aplicando el mismo argumento al resto de los términos, la siguiente prioridad sería entonces el término con peso $w_1$, por lo que $U_{V}(\vec{\theta})|\rho_1 \rangle$ sería el estado propio correspondiente a $\lambda_1$, y los demás términos estarían contenidos en el espacio propio de {$\lambda_2,...,\lambda_{N-1}$}.

Al razonar inductivamente, deducimos que $U_{V}(\vec{\theta})|\rho_j \rangle$ será un estado propio aproximado de $\lambda_j$ para $0\leq j &lt; k$.

Los SSVQE se pueden resumir de la siguiente manera:

- Prepare varios estados de referencia aplicando un U_R unitario a k estados de base computacional diferentes
    - Este algoritmo requiere el uso de $k$ estados de referencia mutuamente ortogonales `{latex} \{ |\rho_j\rangle \}_{j=0}^{k-1}` , tal que $\langle \rho_j | \rho_l \rangle = \delta_{jl}$ para $j,l&lt;k$.
- Aplique la forma variacional $U_V(\vec\theta_{i,j})$ a cada estado de referencia, dando como resultado el siguiente ansatz $U_A(\vec\theta_{i,j})$.
- Bootstrap en $i=0$ si hay un problema similar disponible (generalmente se encuentra mediante simulación clásica o muestreo).
- Evalúa la función de costo $C(\vec\theta_{i,j}) := \sum_{j=0}^{k-1} w_j \langle \psi_{j}(\vec{\theta}) | \sombrero{H} | \psi_{j}(\vec{\theta}) \rangle$ para todos los estados preparados en una computadora cuántica.
    - Esto se puede dividir en el cálculo del valor esperado para un $\langle \psi_{j}(\vec{\theta}) | \sombrero{H} | \psi_{j}(\vec{\theta}) \rangle$ y multiplicando ese resultado por $w_j$.
    - Luego, la función de costo devuelve la suma de todos los valores esperados ponderados.
- Utilice un optimizador clásico para determinar el siguiente conjunto de parámetros $\Theta_{i+1}$.
- Repita los pasos anteriores hasta lograr la convergencia.

¡Reconstruirá la función de costo de SSVQE en la evaluación!

## Deflación cuántica variacional (VQD)

[VQD](https://arxiv.org/abs/1805.08138) es un método iterativo que extiende VQE para obtener los primeros $k$ valores propios de un $\hat{H}$ observable con valores propios {$\lambda_0, \lambda_1,...,\lambda_{N-1}$}, donde $N\geq k$, en lugar de solo el primero. Para el resto de esta sección, supondremos, sin pérdida de generalidad, que $\lambda_0\leq\lambda_1\leq...\leq\lambda_{N-1}$. VQD introduce la noción de un costo de penalización para guiar el proceso de optimización.

![VQD](images/instances_VQD.png)

VQD introduce un término de penalización, denominado $\beta$, para equilibrar la contribución de cada término superpuesto al costo. Este término de penalización sirve para penalizar el proceso de optimización si no se logra la ortogonalidad. Imponemos esta restricción porque los estados propios de un observable, o un operador hermitiano, correspondientes a diferentes valores propios son siempre ortogonales entre sí, o se puede hacer que lo sean en el caso de degeneración o valores propios repetidos. Por lo tanto, al hacer cumplir la ortogonalidad con el estado propio correspondiente a $\lambda_0$, estamos optimizando efectivamente sobre el subespacio que corresponde al resto de los valores propios {$\lambda_1, \lambda_2,..., \lambda_{N-1}$ }. Aquí, $\lambda_1$ es el valor propio más bajo del resto de los valores propios y, por lo tanto, la solución óptima del nuevo problema se puede obtener usando el teorema variacional.

La idea general detrás de VQD es usar VQE como de costumbre para obtener el valor propio más bajo $\lambda_0 := C_0(\vec\theta^0) \equiv C_\text{VQE}(\vec\theta^0)$ junto con el estado propio correspondiente (aproximado) $|\psi(\vec{\theta^0})\rangle$ para algún vector de parámetros óptimo $\vec{\theta^0}$. Luego, para obtener el siguiente valor propio $\lambda_1 &gt; \lambda_0$, en lugar de minimizar la función de costo $C_0(\vec{\theta}) := \langle \psi(\vec{\theta}) | \sombrero{H} | \psi(\vec{\theta})\rangle$, optimizamos:

$$
C_1(\vec{\theta}) :=
C_0(\vec{\theta})+ \beta_0 |\langle \psi(\vec{\theta})| \psi(\vec{\theta^0})\rangle  |^2
$$

El valor positivo $\beta_0$ idealmente debería ser mayor que $\lambda_1-\lambda_0$.

Esto introduce una nueva función de costo que puede verse como un problema restringido, donde minimizamos $C_\text{VQE}(\vec{\theta}) = \langle \psi(\vec{\theta}) | \sombrero{H} | \psi(\vec{\theta})\rangle$ sujeto a la restricción de que el estado debe ser ortogonal al $|\psi(\vec{\theta^0})\rangle$ obtenido previamente, con $\beta_0$ actuando como un término de penalización si la restricción no se cumple.

Alternativamente, este nuevo problema puede interpretarse como la ejecución de VQE en el nuevo observable:

$$
\hat{H_1} := \hat{H} + \beta_0 |\psi(\vec{\theta^0})\rangle \langle \psi(\vec{\theta^0})|
\quad \Rightarrow \quad
C_1(\vec{\theta}) = \langle \psi(\vec{\theta}) | \hat{H_1} | \psi(\vec{\theta})\rangle,
$$

Suponiendo que la solución al nuevo problema es $|\psi(\vec{\theta^1})\rangle$, el valor esperado de $\hat{H}$ (no $\hat{H_1}$) debería ser $ \langle \psi(\vec{\theta^1}) | \sombrero{H} | \psi(\vec{\theta^1})\rangle = \lambda_1$.

Para obtener el tercer autovalor $\lambda_2$, la función de costo a optimizar es:

$$
C_2(\vec{\theta}) :=
C_1(\vec{\theta}) + \beta_1 |\langle \psi(\vec{\theta})| \psi(\vec{\theta^1})\rangle  |^2
$$

donde $\beta_1$ es una constante positiva lo suficientemente grande como para imponer la ortogonalidad del estado de la solución a $|\psi(\vec{\theta^0})\rangle$ y $|\psi(\vec{\theta^1 })\ángulo$. Esto penaliza a los estados en el espacio de búsqueda que no cumplen con este requisito, restringiendo efectivamente el espacio de búsqueda. Por tanto, la solución óptima del nuevo problema debería ser el estado propio correspondiente a $\lambda_2$.

Al igual que el caso anterior, este nuevo problema también puede interpretarse como VQE con el observable:

$$
\hat{H_2} := \hat{H_1} + \beta_1 |\psi(\vec{\theta^1})\rangle \langle \psi(\vec{\theta^1})|
\quad \Rightarrow \quad
C_2(\vec{\theta}) = \langle \psi(\vec{\theta}) | \hat{H_2} | \psi(\vec{\theta})\rangle.
$$

Si la solución a este nuevo problema es $|\psi(\vec{\theta^2})\rangle$, el valor esperado de $\hat{H}$ (no $\hat{H_2}$) debería ser $ \langle \psi(\vec{\theta^2}) | \sombrero{H} | \psi(\vec{\theta^2})\rangle = \lambda_2$.

Análogamente, para obtener el valor propio $k$-th $\lambda_{k-1}$, minimizaría la función de costo:

$$
C_{k-1}(\vec{\theta}) :=
C_{k-2}(\vec{\theta}) + \beta_{k-2} |\langle \psi(\vec{\theta})| \psi(\vec{\theta^{k-2}})\rangle  |^2,
$$

Recuerda que definimos $\vec{\theta^j}$ tal que $\langle \psi(\vec{\theta^j}) | \sombrero{H} | \psi(\vec{\theta^j})\rangle = \lambda_j, \forall j&lt;k$. Este problema es equivalente a minimizar $C(\vec{\theta}) = \langle \psi(\vec{\theta}) | \sombrero{H} | \psi(\vec{\theta})\rangle$ pero con la restricción de que el estado debe ser ortogonal a $|\psi(\vec{\theta^j})\rangle ; \forall j \in {0, \cdots, k-1}$, restringiendo así el espacio de búsqueda al subespacio correspondiente a los valores propios {$\lambda_{k-1},\cdots,\lambda_{N-1}$ }.

Este problema es equivalente a un VQE con el observable:

$$
\hat{H}_{k-1} :=
\hat{H}_{k-2} + \beta_{k-2} |\psi(\vec{\theta^{k-2}})\rangle \langle \psi(\vec{\theta^{k-2}})|
\quad \Rightarrow \quad
C_{k-1}(\vec{\theta}) = \langle \psi(\vec{\theta}) | \hat{H}_{k-1} | \psi(\vec{\theta})\rangle,
$$

Como puede ver en el proceso, para obtener el valor propio $k$-th, necesita los estados propios (aproximados) de los valores propios anteriores de $k-1$, por lo que necesitaría ejecutar VQE un total de $k$ veces. Por lo tanto, la función de costo de VQD es la siguiente:

$$
C_k(\vec{\theta}) =
\langle \psi(\vec{\theta}) | \hat{H} | \psi(\vec{\theta})\rangle +
\sum_{j=0}^{k-1}\beta_j |\langle \psi(\vec{\theta})| \psi(\vec{\theta^j})\rangle |^2
$$

El diseño de VQD se puede resumir de la siguiente manera:

- Preparar un operador de referencia $U_R$
- Aplique la forma variacional $U_V(\vec\theta_{i,j})$ al estado de referencia, creando el siguiente ansatze $U_A(\vec\theta_{i,j})$
- Bootstrap en $i=0$ si tenemos un problema similar (típicamente encontrado a través de simulación clásica o muestreo).
- Evalúe la función de costo $C_k(\vec{\theta})$, que implica calcular $k$ estados excitados y una matriz de $\beta$ que definen la penalización por superposición para cada término superpuesto.
    - Calcule el valor esperado para un $\langle \psi_{j}(\vec{\theta}) | \sombrero{H} | \psi_{j}(\vec{\theta}) \rangle$ para cada $k$
    - Calcula la penalización $\sum_{j=0}^{k-1}\beta_j |\langle \psi(\vec{\theta})| \psi(\vec{\theta^j})\rangle |^2$.
    - La función de costo debe devolver la suma de estos dos términos
- Utilice un optimizador clásico para elegir el siguiente conjunto de parámetros $\Theta_{i+1}$.
- Repita este proceso hasta que se alcance la convergencia.

## Regresión de muestreo cuántico (QSR)

Uno de los principales problemas con VQE son las múltiples llamadas a una computadora cuántica que se requieren para obtener los parámetros para cada paso, incluidos $k$, $k-1$, etc. Esto es especialmente problemático cuando el acceso a los dispositivos cuánticos está en cola. . Si bien una [`Session`](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/how_to/run_session.html) se puede usar para agrupar varias llamadas iterativas, un enfoque alternativo es usar el muestreo. Al utilizar recursos más clásicos, podemos completar el proceso de optimización completo en una sola llamada. Aquí es donde entra en juego [la regresión de muestreo cuántico](https://arxiv.org/pdf/2012.02338) . Dado que el acceso a las computadoras cuánticas sigue siendo un producto de baja oferta/alta demanda, encontramos que esta compensación es posible y conveniente para muchos estudios actuales. Este enfoque aprovecha todas las capacidades clásicas disponibles al mismo tiempo que captura muchos de los mecanismos internos y las propiedades intrínsecas de los cálculos cuánticos que no aparecen en la simulación.

![QSR](images/instances_QSR.png)

La idea detrás de QSR es que la función de costo $C(\theta) := \langle \psi(\theta) | \sombrero{H} | \psi(\theta)\rangle$ se puede expresar como una serie de Fourier de la siguiente manera:

$$
\begin{aligned}
C(\vec{\theta})

&amp; := \langle \psi(\theta) | \hat{H} | \psi(\theta)\rangle \\[1mm]

&amp; := a_0 + \sum_{k=1}^S[a_k\cos(k\theta)+ b_k\sin(k\theta)] \\[1mm]

\end{aligned}
$$

Dependiendo de la periodicidad y el ancho de banda de la función original, el conjunto $S$ puede ser finito o infinito. Para los propósitos de esta discusión, supondremos que es infinito. El siguiente paso es muestrear la función de costo $C(\theta)$ varias veces para obtener los coeficientes de Fourier ${a_0, a_k, b_k}_{k=1}^S$. Específicamente, dado que tenemos $2S+1$ de incógnitas, necesitaremos muestrear la función de costo $2S+1$ veces.

Si luego muestreamos la función de costo para los valores de parámetro $2S+1$ {$\theta_1,...,\theta_{2S+1}$}, podemos obtener el siguiente sistema:

$$
\begin{pmatrix} 1 &amp; \cos(\theta_1) &amp; \sin(\theta_1) &amp; \cos(2\theta_1) &amp; ... &amp; \sin(S\theta_1) \\
1 &amp; \cos(\theta_2) &amp; \sin(\theta_2) &amp; \cos(2\theta_2) &amp; \cdots &amp; \sin(S\theta_2)\\
\vdots &amp; \vdots &amp; \vdots &amp; \vdots &amp; \ddots &amp; \vdots\\
1 &amp; \cos(\theta_{2S+1}) &amp; \sin(\theta_{2S+1}) &amp; \cos(2\theta_{2S+1}) &amp; \cdots &amp; \sin(S\theta_{2S+1})
\end{pmatrix} \begin{pmatrix} a_0 \\ a_1 \\ b_1 \\ a_2 \\ \vdots \\ b_S \end{pmatrix} = \begin{pmatrix} C(\theta_1) \\ C(\theta_2) \\ \vdots \\ C(\theta_{2S+1}) \end{pmatrix},
$$

que reescribiremos como

$$
Fa=c.
$$

En la práctica, este sistema generalmente no es consistente porque los valores de la función de costo $c$ no son exactos. Por lo tanto, suele ser una buena idea normalizarlos multiplicándolos por $F^\dagger$ a la izquierda, lo que da como resultado:

$$
F^\dagger Fa = F^\dagger c.
$$

Este nuevo sistema siempre es consistente y su solución es una solución de mínimos cuadrados al problema original. Si tenemos $k$ parámetros en lugar de uno solo, y cada parámetro $\theta^i$ tiene su propio $S_i$ para $i \in {1,...,k}$, entonces el número total de muestras requeridas es:

$$
T=\prod_{i=1}^k(2S_i+1)\leq \prod_{i=1}^k(2S_{max}+1) = (2S_{max}+1)^n,
$$

donde $S_{\max} = \max_i(S_i)$. Además, ajustar $S_{\max}$ como un parámetro ajustable (en lugar de inferirlo) abre nuevas posibilidades, como:

- **Sobremuestreo** : para mejorar la precisión.
- **Submuestreo** : para aumentar el rendimiento al reducir la sobrecarga del tiempo de ejecución o eliminar los mínimos locales.

El diseño de QSR se puede resumir de la siguiente manera:

- Preparar operadores de referencia $U_R$
    - Pasaremos del estado $|0\rangle$ al estado de referencia $|\rho\rangle$
- Aplique la forma variacional $U_V(\vec\theta_{i,j})$ para crear un ansatz $U_A(\vec\theta_{i,j})$
    - Determine el ancho de banda asociado con cada parámetro en el ansatz. Un límite superior es suficiente.
- Bootstrap en $i=0$ si tenemos un problema similar (generalmente encontrado a través de simulación clásica o muestreo)
- Muestra la función de costo $C(\vec\theta) := a_0 + \sum_{k=1}^S[a_k\cos(k\theta)+ b_k\sin(k\theta)]$ al menos $T$ veces
    - $T=\prod_{i=1}^k(2S_i+1)\leq \prod_{i=1}^k(2S_{máx}+1) = (2S_{máx}+1)^n$
    - Decida si sobremuestrear o submuestrear para equilibrar la velocidad frente a la precisión ajustando $T$.
- Calcule los coeficientes de Fourier a partir de las muestras (es decir, resuelva el sistema lineal de ecuaciones normalizado).
- Resuelva el mínimo global de la función de regresión resultante en una máquina clásica.

Con esta lección, aprendió acerca de las múltiples instancias variacionales disponibles:

- Disposición general
- Introducción de ponderaciones y penalizaciones para ajustar una función de coste
- Exploración del submuestreo frente al sobremuestreo para equilibrar la velocidad frente a la precisión

Estas ideas se pueden adaptar para formar un algoritmo variacional personalizado que se ajuste a su problema. Le animamos a compartir sus resultados con la comunidad. La siguiente lección explorará cómo usar un algoritmo variacional para resolver una aplicación.